In [1]:
import pandas as pd
import os

In [5]:
class company_prerocessing():
#     def __init__()
    # 파일 열기
    def open_df(folder_path):
        df = pd.read_csv(folder_path)
        return df

    # 재무 상태표
    def bs_preprocessing(df_bs, cmp_list):

        if df_bs.columns[2].isdigit() == True:
            try:
                df_bs_1 = df_bs.iloc[:, [0, 2, 3, 4]]
            except:
                pass

        elif df_bs.columns[6].isdigit() == True:
            df_bs_1 = df_bs.iloc[:, [1, 6, 7, 8]]

        elif df_bs.columns[7].isdigit() == True:
            df_bs_1 = df_bs.iloc[:, [1, 7, 8, 9]]

        elif df_bs.columns[8].isdigit() == True:
            df_bs_1 = df_bs.iloc[:, [1, 8, 9, 10]]

        else:
            df_bs_1 = df_bs.iloc[:, 0:4]

        df_bs_1.columns = ['', df_bs_1.columns[1], df_bs_1.columns[2], df_bs_1.columns[3]]
        df_bs_1 = df_bs_1.T

        df_bs_1 = df_bs_1.rename(columns=df_bs_1.iloc[0])
        df_bs_1 = df_bs_1.drop(df_bs_1.index[0])
        df_bs_1.drop('label_ko', inplace=True, axis =1)

        df_bs_1 = df_bs_1.reset_index().rename(columns={'index': 'date'})


        df_bs_1['company'] = cmp_list
        print('bs_preprocessing complete')

        return df_bs_1

    # 포괄손익 계산서
    def cis_preprocessing(df_cis, cmp_list):
        
        if '01-20' in df_cis.columns[6]:
            if '01-20' in df_cis.columns[8]:
                df_cis_1 = df_cis.iloc[:, [1, 6, 7, 8]]
            else:
                df_cis_1 = df_cis.iloc[:, [1, 6, 7]]

        else:
            if '01-20' in df_cis.columns[9]:
                df_cis_1 = df_cis.iloc[:, [1, 7, 8, 9]]
            else:
                df_cis_1 = df_cis.iloc[:, [1, 7, 8]]

        if len(df_cis_1.columns) == 4:
            df_cis_1.columns = ['', df_cis_1.columns[1], df_cis_1.columns[2], df_cis_1.columns[3]]
        elif len(df_cis_1.columns) == 3:
            df_cis_1.columns = ['', df_cis_1.columns[1], df_cis_1.columns[2]]

        df_cis_1 = df_cis_1.T

        df_cis_1 = df_cis_1.rename(columns=df_cis_1.iloc[0])
        df_cis_1 = df_cis_1.drop(df_cis_1.index[0])
        df_cis_1.drop('label_ko', inplace=True, axis =1)
        
        df_cis_1 = df_cis_1.reset_index().rename(columns={'index': 'date'})

        df_cis_1['company'] = cmp_list
        df_cis_1['date'] = df_cis_1['date'].str[9:]
        print('cis_preprocessing complete')

        return df_cis_1
    # 데이터 프레임 저장
    def save_df(folder_path, df, cmp_list):
        df.to_csv(folder_path + cmp_list + '.csv', index=False)

In [4]:
# if '__main__' == __name__:

FOLDER_PATH = 'data/'
# 원본 PATH
PATH1 = '최종/건실기업_재무상태표/'
PATH2 = '최종/건실기업_포괄손익계산서/'

#전처리 PATH
PATH3 = '최종/건실기업_재무상태표 전처리/'
PATH4 = '최종/건실기업_포괄손익계산서 전처리/'

# 파일 리스트화
file_list1 = os.listdir(FOLDER_PATH + PATH1)
file_list2 = os.listdir(FOLDER_PATH + PATH2)

var_1_dic = {'자본총계': 4, '자산총계': 4, '매입채무': 13, '매출채권': 13, '부채총계': 4, '유동부채': 4, '유동자산': 4, '재고자산': 4}
var_2_dic = {'매출총이익': 5, '법인세비용차감': 15, '당기순이익': 10}

bs_a = pd.DataFrame(columns=['date', 'company'])
cis_a = pd.DataFrame(columns=['date', 'company'])

for cmp_list1, cmp_list2 in zip(file_list1, file_list2):
    print(cmp_list1, cmp_list2)
    # 재무 상태표
    df = company_prerocessing.open_df(os.path.join(FOLDER_PATH + PATH1 + cmp_list1))
    df_bs_1 = company_prerocessing.bs_preprocessing(df, cmp_list1[:-13])
    company_prerocessing.save_df(os.path.join(FOLDER_PATH + PATH3), df_bs_1, cmp_list1[:-13])

    # 변수 전처리
    list1 = ['date', 'company']
    for var1, var1_ in var_1_dic.items():
        for bs in df_bs_1.columns:
            if var1 in bs:
                if var1_ == len(bs):
                    list1.append(bs)
    print(df_bs_1[list1])

    bs_a = pd.concat([bs_a, df_bs_1[list1]])

    # 포괄손익 계산서
    df = company_prerocessing.open_df(os.path.join(FOLDER_PATH + PATH2+ cmp_list2))
    df_cis_1 = company_prerocessing.cis_preprocessing(df, cmp_list2[:-15])
    company_prerocessing.save_df(os.path.join(FOLDER_PATH + PATH4), df_cis_1, cmp_list2[:-15])

    # 변수 전처리
    list2 = ['date', 'company']
    for var2, var2_ in var_2_dic.items():
        for cis in df_cis_1.columns:
            if var2 in cis:
                if var2_ >= len(cis):
                    list2.append(cis)
    print(df_cis_1[list2])
    cis_a = pd.concat([cis_a, df_cis_1[list2]])



bs_a.reset_index(inplace=True, drop=True)
cis_a.reset_index(inplace=True, drop=True)
company_prerocessing.save_df(os.path.join(FOLDER_PATH + PATH3), bs_a, '재무상태표 통합')
company_prerocessing.save_df(os.path.join(FOLDER_PATH + PATH4), cis_a, '포괄손익계산서 통합')


DMS 재무상태표 원본.csv DMS 포괄손익계산서 원본.csv
bs_preprocessing complete
       date company            자본총계            자산총계  매입채무 및 기타유동채무  \
0  20191231     DMS  166005674664.0  302514623836.0  32096890971.0   
1  20181231     DMS  151479426195.0  317314698057.0  39977684190.0   
2  20171231     DMS  135202873362.0  298505199522.0  69184418522.0   

   매출채권 및 기타유동채권            부채총계            유동부채            유동자산  \
0  26620302586.0  136508949172.0  135345962490.0  134517297914.0   
1  47088416662.0  165835271862.0  162426446113.0  159939971787.0   
2  25987507374.0  163302326160.0  146972021897.0  149549936171.0   

            재고자산  
0  14498465305.0  
1  14616273960.0  
2  17969335446.0  
cis_preprocessing complete
       date company          매출총이익 법인세비용차감전순이익(손실)      당기순이익(손실)
0  20191231     DMS  65300086704.0   21544765028.0  17236232679.0
1  20181231     DMS  75967702026.0   21868026243.0  15184128518.0
2  20171231     DMS  61208001998.0    4835649098.0   3857602202.0
SGA 재무상태표 원본.csv 

In [6]:
pd.merge(cis_a, bs_a, on=['date','company'])


[]

In [8]:
bs_a

,date,company,자본총계,자산총계,매입채무 및 기타유동채무,매출채권 및 기타유동채권,부채총계,유동부채,유동자산,재고자산
0,20191231,DMS,166005674664.0,302514623836.0,32096890971.0,26620302586.0,136508949172.0,135345962490.0,134517297914.0,14498465305.0
1,20181231,DMS,151479426195.0,317314698057.0,39977684190.0,47088416662.0,165835271862.0,162426446113.0,159939971787.0,14616273960.0
2,20171231,DMS,135202873362.0,298505199522.0,69184418522.0,25987507374.0,163302326160.0,146972021897.0,149549936171.0,17969335446.0
3,20191231,SGA,38180144932.0,100397309576.0,18995167756.0,19748688241.0,62217164644.0,56031358178.0,40092142943.0,3037803693.0
4,20181231,SGA,68827310603.0,132956451118.0,20527160409.0,34032856006.0,64129140515.0,40498391054.0,72299333082.0,8000743477.0
5,20171231,SGA,78939336234.0,147318939743.0,26793245616.0,47424205868.0,68379603509.0,36894397389.0,81394442916.0,11783790589.0
6,20191231,메디톡스,262680764325.0,421529503965.0,24996604550.0,61270483065.0,158848739640.0,136347752994.0,120029581705.0,29332108117.0
7,20181231,메디톡스,257241639872.0,395014965909.0,9653147639.0,67356314464.0,137773326037.0,110815703919.0,116431673406.0,21510965453.0
8,20171231,메디톡스,196525985189.0,380945209080.0,10974877525.0,56003260656.0,184419223891.0,137389118578.0,103154147912.0,11609096023.0
9,20191231,명성티엔에스,38250153597.0,66256203865.0,3206398527.0,11354382837.0,28006050268.0,25743128061.0,38207439387.0,15489115612.0


In [9]:
cis_a

,date,company,매출총이익,법인세비용차감전순이익(손실),당기순이익(손실),자본총계,자산총계,매입채무 및 기타유동채무,매출채권 및 기타유동채권,부채총계,유동부채,유동자산,재고자산
0,20191231,DMS,65300086704.0,21544765028.0,17236232679.0,166005674664.0,302514623836.0,32096890971.0,26620302586.0,136508949172.0,135345962490.0,134517297914.0,14498465305.0
1,20181231,DMS,75967702026.0,21868026243.0,15184128518.0,151479426195.0,317314698057.0,39977684190.0,47088416662.0,165835271862.0,162426446113.0,159939971787.0,14616273960.0
2,20171231,DMS,61208001998.0,4835649098.0,3857602202.0,135202873362.0,298505199522.0,69184418522.0,25987507374.0,163302326160.0,146972021897.0,149549936171.0,17969335446.0
3,20191231,SGA,16251524737.0,-29082118459.0,-30354499317.0,38180144932.0,100397309576.0,18995167756.0,19748688241.0,62217164644.0,56031358178.0,40092142943.0,3037803693.0
4,20181231,SGA,16251573373.0,-15167792417.0,-14341575045.0,68827310603.0,132956451118.0,20527160409.0,34032856006.0,64129140515.0,40498391054.0,72299333082.0,8000743477.0
5,20171231,SGA,24189006614.0,-17163753103.0,-17768449236.0,78939336234.0,147318939743.0,26793245616.0,47424205868.0,68379603509.0,36894397389.0,81394442916.0,11783790589.0
6,20191231,메디톡스,136370349140.0,25149968451.0,25563082881.0,262680764325.0,421529503965.0,24996604550.0,61270483065.0,158848739640.0,136347752994.0,120029581705.0,29332108117.0
7,20181231,메디톡스,151206884234.0,85099485013.0,69765654196.0,257241639872.0,395014965909.0,9653147639.0,67356314464.0,137773326037.0,110815703919.0,116431673406.0,21510965453.0
8,20171231,메디톡스,146647432432.0,85247217489.0,69993572739.0,196525985189.0,380945209080.0,10974877525.0,56003260656.0,184419223891.0,137389118578.0,103154147912.0,11609096023.0
9,20191231,명성티엔에스,6912545346.0,726559696.0,504333074.0,38250153597.0,66256203865.0,3206398527.0,11354382837.0,28006050268.0,25743128061.0,38207439387.0,15489115612.0


In [28]:
df = company_prerocessing.open_df(os.path.join(FOLDER_PATH + PATH1 + file_list1[0]))
df_bs_1 = company_prerocessing.bs_preprocessing(df, file_list1[0][:-13])

bs_preprocessing complete


In [37]:
list1 = ['date', 'company']
for var1, var1_ in var_1_dic.items():
    for bs in df_bs_1.columns:
        if var1 in bs:
            if var1_ == len(bs):
                list1.append(bs)
                print(bs)

자본총계
자산총계
매입채무 및 기타유동채무
매출채권 및 기타유동채권
부채총계
유동부채
유동자산
재고자산


In [38]:
df_bs_1[list1]


,date,company,자본총계,자산총계,매입채무 및 기타유동채무,매출채권 및 기타유동채권,부채총계,유동부채,유동자산,재고자산
0,20191231,DMS,166005674664.0,302514623836.0,32096890971.0,26620302586.0,136508949172.0,135345962490.0,134517297914.0,14498465305.0
1,20181231,DMS,151479426195.0,317314698057.0,39977684190.0,47088416662.0,165835271862.0,162426446113.0,159939971787.0,14616273960.0
2,20171231,DMS,135202873362.0,298505199522.0,69184418522.0,25987507374.0,163302326160.0,146972021897.0,149549936171.0,17969335446.0


In [39]:
list1 = ['date', 'company']
for var1, var1_ in var_1_dic.items():
    for bs in df_bs_1.columns:
        if var1 in bs:
            if var1_ >= len(bs):
                list1.append(bs)
                print(bs)

자본총계
자산총계
매입채무 및 기타유동채무
장기매입채무및기타채무
매출채권 및 기타유동채권
장기성매출채권및기타채권
부채총계
유동부채
유동자산
재고자산


In [52]:
len('단기매입채무 및 기타채무')

13

In [53]:
len('매입채무 및 기타유동채무')

13

In [56]:
if '단기매입채무 및 기타채무' in bs_a.columns:
    bs_a.drop('단기매입채무 및 기타채무', axis=1)

,date,company,자본총계,자산총계,매입채무 및 기타유동채무,매출채권 및 기타유동채권,부채총계,유동부채,유동자산,재고자산
0,20191231,DMS,166005674664.0,302514623836.0,32096890971.0,26620302586.0,136508949172.0,135345962490.0,134517297914.0,14498465305.0
1,20181231,DMS,151479426195.0,317314698057.0,39977684190.0,47088416662.0,165835271862.0,162426446113.0,159939971787.0,14616273960.0
2,20171231,DMS,135202873362.0,298505199522.0,69184418522.0,25987507374.0,163302326160.0,146972021897.0,149549936171.0,17969335446.0
3,20191231,SGA,38180144932.0,100397309576.0,18995167756.0,19748688241.0,62217164644.0,56031358178.0,40092142943.0,3037803693.0
4,20181231,SGA,68827310603.0,132956451118.0,20527160409.0,34032856006.0,64129140515.0,40498391054.0,72299333082.0,8000743477.0
5,20171231,SGA,78939336234.0,147318939743.0,26793245616.0,47424205868.0,68379603509.0,36894397389.0,81394442916.0,11783790589.0
6,20191231,메디톡스,262680764325.0,421529503965.0,24996604550.0,61270483065.0,158848739640.0,136347752994.0,120029581705.0,29332108117.0
7,20181231,메디톡스,257241639872.0,395014965909.0,9653147639.0,67356314464.0,137773326037.0,110815703919.0,116431673406.0,21510965453.0
8,20171231,메디톡스,196525985189.0,380945209080.0,10974877525.0,56003260656.0,184419223891.0,137389118578.0,103154147912.0,11609096023.0
9,20191231,명성티엔에스,38250153597.0,66256203865.0,3206398527.0,11354382837.0,28006050268.0,25743128061.0,38207439387.0,15489115612.0
